In [2]:
f = open("schedule.txt","r")
lines = f.read().splitlines()

In [15]:
sem = input("Enter semester: ")
if(sem == 'fall'):
    sem_start = '2019-09-05'
    sem_end = '20191212'
else:
    sem_start = '2020-01-21'
    sem_end = '20190503'

Enter semester: spring


In [4]:
Monday = lines[lines.index('Monday')+1:lines.index('Tuesday')-1]
Tuesday = lines[lines.index('Tuesday')+1:lines.index('Wednesday')-1]
Wednesday = lines[lines.index('Wednesday')+1:lines.index('Thursday')-1]
Thursday = lines[lines.index('Thursday')+1:lines.index('Friday')-1]
Friday = lines[lines.index('Friday')+1:]
schedule_by_day = [Monday, Tuesday, Wednesday, Thursday, Friday]

In [5]:
master = []
i = -1
for day in schedule_by_day:
    i = i+1
    classes = day[0::5]
    lectures = day[1::5]
    locations = day[2::5]
    times = day[3::5]
    d = ["MO", "TU", "WE", "TH", "FR"]
    for e in range(0, len(classes)):
            a = times[e].split("to")
            a[0] = a[0].strip()
            a[1] = a[1].strip()

            if 'AM' in a[0]:
                start_time = a[0].split(" ")[0]
            elif 'PM' in a[0]:
                start_time = a[0].split(" ")[0]
                if(start_time[0:2]!='12'):
                    start_time = str(int(start_time[0:start_time.index(':')])+12) + start_time[start_time.index(':'):]

            if 'AM' in a[1]:
                end_time = a[1].split(" ")[0]
            elif 'PM' in a[1]:
                end_time = a[1].split(" ")[0]
                if(end_time[0:2]!='12'):
                    end_time = str(int(end_time[0:end_time.index(':')])+12) + end_time[end_time.index(':'):]
                
            event = {
                'summary': classes[e],
                'location': locations[e] + ', Madison, WI 53706',
                'description': lectures[e],
                'start': {
                    'dateTime': sem_start + 'T'+start_time+':00-05:00',
                    'timeZone': 'America/Chicago',
                },
                'end': {
                    'dateTime': sem_start + 'T'+end_time+':00-05:00',
                    'timeZone': 'America/Chicago',
                },
                'recurrence': [
                    'RRULE:FREQ=WEEKLY;UNTIL='+sem_end+';BYDAY='+d[i]
                ],
                'reminders': {
                    'useDefault': False,
                    'overrides': [
                      {'method': 'popup', 'minutes': 15},
                    ],
                },
            }
            master.append(event)

In [12]:
from datetime import datetime, timedelta
from cal_setup import get_calendar_service


def create(event):
    # creates one hour event tomorrow 10 AM IST
    service = get_calendar_service()

    event_result = service.events().insert(calendarId='primary',
        body=event).execute()

    print("created event")
    print("id: ", event_result['id'])
    print("summary: ", event_result['summary'])
    print("starts at: ", event_result['start']['dateTime'])
    print("ends at: ", event_result['end']['dateTime'])

In [14]:
create(master[19])

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=731019933795-39haj4td4n10fqk2aure8t7ha713ue66.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A51810%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=nZX8ZMZWVaiHLO1DFWtJzb2HctxHpb&access_type=offline
created event
id:  1jansd3mgdea4f5fugjq2s0ef0
summary:  E C E 352:  Digital System Fundamentals
starts at:  2019-09-05T11:00:00-05:00
ends at:  2019-09-05T11:50:00-05:00


In [8]:
master[18]

{'summary': 'E C E 352:  Digital System Fundamentals',
 'location': '410 Wendt Commons, Madison, WI 53706',
 'description': 'LEC 001',
 'start': {'dateTime': '2019-09-05T11:00:00-05:00',
  'timeZone': 'America/Chicago'},
 'end': {'dateTime': '2019-09-05T11:50:00-05:00',
  'timeZone': 'America/Chicago'},
 'recurrence': ['RRULE:FREQ=WEEKLY;UNTIL=20191212;BYDAY=FR'],
 'reminders': {'useDefault': False,
  'overrides': [{'method': 'popup', 'minutes': 15}]}}

In [ ]:
type(day)